# Forecasting Weather with Neural Prophet

## Import Dependencies

In [1]:
import pandas as pd
from neuralprophet import NeuralProphet
from matplotlib import pyplot as plt
import numpy as np
import os

## Load the dataset and process data

In [2]:
# Define the path to the dataset
dataset_path = "../DataPreprocessing/Dataset/SriLanka_Weather_Dataset new.csv"

# Load the dataset into a Pandas DataFrame
weather_df = pd.read_csv(dataset_path)

print("Dataset Information:")
print(weather_df.info())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147480 entries, 0 to 147479
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   time                        147480 non-null  object 
 1   weathercode                 147480 non-null  int64  
 2   temperature_2m_max          147480 non-null  float64
 3   temperature_2m_min          147480 non-null  float64
 4   temperature_2m_mean         147480 non-null  float64
 5   apparent_temperature_max    147480 non-null  float64
 6   apparent_temperature_min    147480 non-null  float64
 7   apparent_temperature_mean   147480 non-null  float64
 8   sunrise                     147480 non-null  object 
 9   sunset                      147480 non-null  object 
 10  shortwave_radiation_sum     147480 non-null  float64
 11  precipitation_sum           147480 non-null  float64
 12  rain_sum                    147480 non-null  float6

In [3]:
weather_df.city.unique()

array(['Colombo', 'Mount Lavinia', 'Kesbewa', 'Moratuwa', 'Maharagama',
       'Ratnapura', 'Kandy', 'Negombo', 'Sri Jayewardenepura Kotte',
       'Kalmunai', 'Trincomalee', 'Galle', 'Jaffna', 'Athurugiriya',
       'Weligama', 'Matara', 'Kolonnawa', 'Gampaha', 'Puttalam',
       'Badulla', 'Kalutara', 'Bentota', 'Matale', 'Mannar', 'Pothuhera',
       'Kurunegala', 'Mabole', 'Hatton', 'Hambantota', 'Oruwala'],
      dtype=object)

In [4]:
# Convert 'time' to datetime
weather_df['time'] = pd.to_datetime(weather_df['time'])

# Convert 'sunrise' and 'sunset' columns to datetime format and extract only time
weather_df['sunrise'] = pd.to_datetime(weather_df['sunrise']).dt.time
weather_df['sunset'] = pd.to_datetime(weather_df['sunset']).dt.time
weather_df['sunrise'] = pd.to_datetime(weather_df['sunrise'], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(weather_df['sunrise'], format='%H:%M:%S').dt.minute
weather_df['sunset'] = pd.to_datetime(weather_df['sunset'], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(weather_df['sunset'], format='%H:%M:%S').dt.minute

# Rename 'precipitation_hours' to 'rain_hours'
weather_df.rename(columns={'precipitation_hours': 'rain_hours'}, inplace=True)

# Remove unnecessary columns
columns_to_remove = ['precipitation_sum', 'snowfall_sum', 'latitude', 'longitude', 'elevation', 'country', 'temperature_2m_mean', 'apparent_temperature_mean']
weather_df.drop(columns=columns_to_remove, inplace=True)

# Reorder columns
column_order = ['time', 'city', 'weathercode', 'temperature_2m_max', 'temperature_2m_min', 
                'apparent_temperature_max', 'apparent_temperature_min',
                'sunrise', 'sunset', 'shortwave_radiation_sum',
                'rain_sum', 'rain_hours', 'windspeed_10m_max', 'windgusts_10m_max',
                'winddirection_10m_dominant', 'et0_fao_evapotranspiration']

# Rearrange columns
weather_df = weather_df[column_order]

# Display the first few rows of the modified dataset
weather_df.head()

,time,city,weathercode,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,shortwave_radiation_sum,rain_sum,rain_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration
0,2010-01-01,Colombo,2,30.0,22.7,34.4,25.2,52,755,20.92,0.0,0,11.7,27.4,20,4.58
1,2010-01-02,Colombo,51,29.9,23.5,33.8,26.2,52,756,17.71,0.1,1,13.0,27.0,24,3.84
2,2010-01-03,Colombo,51,29.5,23.2,34.3,26.3,53,756,17.76,0.6,3,12.3,27.4,16,3.65
3,2010-01-04,Colombo,2,28.9,21.9,31.6,23.4,53,757,16.50,0.0,0,17.0,34.6,356,3.79
4,2010-01-05,Colombo,1,28.1,21.3,30.1,23.1,53,757,23.61,0.0,0,18.7,37.1,355,4.97


In [5]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147480 entries, 0 to 147479
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   time                        147480 non-null  datetime64[ns]
 1   city                        147480 non-null  object        
 2   weathercode                 147480 non-null  int64         
 3   temperature_2m_max          147480 non-null  float64       
 4   temperature_2m_min          147480 non-null  float64       
 5   apparent_temperature_max    147480 non-null  float64       
 6   apparent_temperature_min    147480 non-null  float64       
 7   sunrise                     147480 non-null  int64         
 8   sunset                      147480 non-null  int64         
 9   shortwave_radiation_sum     147480 non-null  float64       
 10  rain_sum                    147480 non-null  float64       
 11  rain_hours                  147480 non-

In [6]:
weather_df.isnull().sum()

time                          0
city                          0
weathercode                   0
temperature_2m_max            0
temperature_2m_min            0
apparent_temperature_max      0
apparent_temperature_min      0
sunrise                       0
sunset                        0
shortwave_radiation_sum       0
rain_sum                      0
rain_hours                    0
windspeed_10m_max             0
windgusts_10m_max             0
winddirection_10m_dominant    0
et0_fao_evapotranspiration    0
dtype: int64

## Test the Code

In [7]:
# Get unique locations
locations = weather_df['city'].unique()[:2]  # Use the first two cities for testing

# Create main output directory if it doesn't exist
output_directory = "WeatherForecasts"
os.makedirs(output_directory, exist_ok=True)

# Initialize location_models and location_forecasts dictionaries
location_models = {}
location_forecasts = {}

# Loop through each location
for location in locations:
    print(f"Training models for {location}...")
    
    # Create subdirectory for the current location
    location_output_directory = os.path.join(output_directory, location)
    os.makedirs(location_output_directory, exist_ok=True)
    
    # Filter data for the specific location
    location_data = weather_df[weather_df['city'] == location]
    
    # Rename columns for NeuralProphet
    location_data.rename(columns={'time': 'ds', 'temperature_2m_max': 'y_max_temp', 'temperature_2m_min': 'y_min_temp'}, inplace=True)
    
    # Initialize models for max and min temperature
    target_vars = ['y_max_temp', 'y_min_temp']
    
    location_models[location] = {}
    location_forecasts[location] = {}
    
    for target_var in target_vars:
        print(f"Training model for {location} - {target_var}...")
        
        # Create a new dataframe for the current target variable
        target_data = location_data[['ds', target_var]].rename(columns={target_var: 'y'})
        
        # Initialize and fit the model for the current target variable
        model = NeuralProphet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True,  # Adjust based on data
        )
        model.fit(target_data, freq='D', epochs=100)
        
        # Save the model for later use
        location_models[location][target_var] = model
        
        # Generate future dataframe
        future = model.make_future_dataframe(target_data, periods=10000)  # Adjust the number of periods as needed
        
        # Make predictions
        forecast = model.predict(future)
        
        # Save the forecast to CSV
        forecast.to_csv(os.path.join(location_output_directory, f"{target_var}_forecast.csv"), index=False)
        
        # Save the forecast in the dictionary for later use
        location_forecasts[location][target_var] = forecast

WARNING - (py.warnings._showwarnmsg) - C:\Users\niwar\AppData\Local\Temp\ipykernel_4052\1024756041.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_data.rename(columns={'time': 'ds', 'temperature_2m_max': 'y_max_temp', 'temperature_2m_min': 'y_min_temp'}, inplace=True)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.98% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


Training models for Colombo...
Training model for Colombo - y_max_temp...


INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (154) is too small than the required number for the learning rate finder (242). The results might not be optimal.


Finding best initial lr:   0%|          | 0/242 [00:00<?, ?it/s]

...

Finding best initial lr:   0%|          | 0/242 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.98% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.99% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.99% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 154it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


-----------------
-----------------

In [14]:
# Function to generate future dataframe based on the model and target data
def generate_future_dataframe(model, target_data, forecast_date):
    # Calculate the number of days between forecast_date and the end date of your data
    days_to_forecast = (forecast_date - pd.to_datetime('2023-06-17')).days
    
    # Generate future dataframe for the entire period until forecast_date
    future = model.make_future_dataframe(target_data, periods=days_to_forecast + 12000)
    
    # Find the index for the forecast_date
    forecast_date_index = future[future['ds'] == forecast_date].index[0]
    
    return future, forecast_date_index


# Function to get predictions for a given location and date
def get_weather_forecast(location, forecast_date, output_directory="WeatherForecasts"):
    # Check if the location and model for the location exist
    if location in location_models:
        # Check if the model for max temperature exists
        if 'y_max_temp' in location_models[location]:
            max_temp_model = location_models[location]['y_max_temp']
            
            # Generate future dataframe for max temperature
            future_max_temp, forecast_date_index_max = generate_future_dataframe(max_temp_model, target_data, forecast_date)
            
            # Make predictions for max temperature starting from the forecast_date
            max_temp_forecast = max_temp_model.predict(future_max_temp.loc[forecast_date_index_max:])
            
            # Save max_temp_forecast to CSV
            max_temp_forecast.to_csv(os.path.join(output_directory, f"{location}_max_temp_forecast.csv"), index=False)
        else:
            max_temp_forecast = None  # No model for max temperature
        
        # Check if the model for min temperature exists
        if 'y_min_temp' in location_models[location]:
            min_temp_model = location_models[location]['y_min_temp']
            
            # Generate future dataframe for min temperature
            future_min_temp, forecast_date_index_min = generate_future_dataframe(min_temp_model, target_data, forecast_date)
            
            # Make predictions for min temperature starting from the forecast_date
            min_temp_forecast = min_temp_model.predict(future_min_temp.loc[forecast_date_index_min:])
            
            # Save min_temp_forecast to CSV
            min_temp_forecast.to_csv(os.path.join(output_directory, f"{location}_min_temp_forecast.csv"), index=False)
        else:
            min_temp_forecast = None  # No model for min temperature
        
        # Return the forecasts for max and min temperatures
        return max_temp_forecast, min_temp_forecast
    
    else:
        print(f"No model found for the location: {location}")
        return None, None


# Example usage
forecast_date = pd.to_datetime('2023-06-18')  # Replace with your desired forecast date
location = 'Colombo'  # Replace with your desired location

# Create output directory if it doesn't exist
output_directory = "WeatherForecasts"
os.makedirs(output_directory, exist_ok=True)

max_temp_forecast, min_temp_forecast = get_weather_forecast(location, forecast_date, output_directory)

# Print or use the forecasts as needed
print("\nMax Temperature Forecast:")
print(max_temp_forecast)

print("\nMin Temperature Forecast:")
print(min_temp_forecast)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.98% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.992% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.992% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 154it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.98% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.992% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.992% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 154it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column



Max Temperature Forecast:
              ds     y      yhat1      trend  season_yearly  season_weekly  \
0     2023-06-18  None  28.529854  32.631657      -0.741373      -0.018990   
1     2023-06-19  None  28.552145  32.630882      -0.747089       0.009791   
2     2023-06-20  None  28.546331  32.630108      -0.752375       0.010038   
3     2023-06-21  None  28.533165  32.629333      -0.757303       0.002574   
4     2023-06-22  None  28.528572  32.628563      -0.761945       0.003395   
...          ...   ...        ...        ...            ...            ...   
11996 2056-04-21  None  21.018930  23.349108       1.029601      -0.018337   
11997 2056-04-22  None  21.008129  23.348333       0.989707       0.011528   
11998 2056-04-23  None  20.936409  23.347561       0.949279      -0.018990   
11999 2056-04-24  None  20.923479  23.346786       0.908341       0.009791   
12000 2056-04-25  None  20.881529  23.346012       0.866919       0.010038   

       season_daily  
0          -3.

---------------

## Original Code For All Locations and Weather Data

In [ ]:
# TODO: Need to Update According to the Model Test code

# Get unique locations
locations = weather_df['city'].unique()

# Initialize an empty dictionary to store models and forecasts for each location
location_models = {}
location_forecasts = {}

# Loop through each location
for location in locations:
    print(f"Training models for {location}...")
    
    # Filter data for the specific location
    location_data = weather_df[weather_df['city'] == location]
    
    # Rename columns for NeuralProphet
    location_data.rename(columns={'time': 'ds', 'temperature_2m_max': 'y_max_temp', 'temperature_2m_min': 'y_min_temp',
                                    'apparent_temperature_max': 'y_apparent_max_temp', 'apparent_temperature_min': 'y_apparent_min_temp',
                                    'rain_sum': 'y_rain_sum', 'rain_hours': 'y_rain_hours',
                                    'sunrise': 'y_sunrise', 'sunset': 'y_sunset',
                                    'shortwave_radiation_sum': 'y_shortwave_radiation_sum',
                                    'windspeed_10m_max': 'y_windspeed_10m_max', 'windgusts_10m_max': 'y_windgusts_10m_max',
                                    'winddirection_10m_dominant': 'y_winddirection_10m_dominant',
                                    'et0_fao_evapotranspiration': 'y_et0_fao_evapotranspiration'}, inplace=True)
    
    # Initialize models for each target variable
    target_vars = ['y_max_temp', 'y_min_temp', 'y_apparent_max_temp', 'y_apparent_min_temp',
                    'y_sunrise', 'y_sunset', 'y_shortwave_radiation_sum', 'y_rain_sum',
                    'y_rain_hours', 'y_windspeed_10m_max', 'y_windgusts_10m_max',
                    'y_winddirection_10m_dominant', 'y_et0_fao_evapotranspiration']
    
    location_models[location] = {}
    location_forecasts[location] = {}
    
    for target_var in target_vars:
        print(f"Training model for {location} - {target_var}...")
        
        # Create a new dataframe for the current target variable
        target_data = location_data[['ds', target_var]].rename(columns={target_var: 'y'})
        
        # Initialize and fit the model for the current target variable
        model = NeuralProphet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True,  # Adjust based on data
        )
        model.fit(target_data, freq='D', epochs=100)
        
        # Save the model for later use
        location_models[location][target_var] = model
        
        # Generate future dataframe
        future = model.make_future_dataframe(target_data, periods=12000)  # Adjust the number of periods as needed
        
        # Make predictions
        forecast = model.predict(future)
        
        # Save the forecast
        location_forecasts[location][target_var] = forecast

WARNING - (py.warnings._showwarnmsg) - C:\Users\niwar\AppData\Local\Temp\ipykernel_37200\1192424720.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_data.rename(columns={'time': 'ds', 'temperature_2m_max': 'y_max_temp', 'temperature_2m_min': 'y_min_temp',

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.98% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


Training models for Colombo...
Training model for Colombo - y_max_temp...


INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (154) is too small than the required number for the learning rate finder (242). The results might not be optimal.


Finding best initial lr:   0%|          | 0/242 [00:00<?, ?it/s]

...

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.98% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


Training model for Colombo - y_sunset...


INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (154) is too small than the required number for the learning rate finder (242). The results might not be optimal.


Finding best initial lr:   0%|          | 0/242 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
# Plot the forecasts for each location
for location, forecast in location_forecasts.items():
    fig = model.plot(forecast)
    plt.title(f"{location} - Weather Forecast")
    plt.show()